

## 🧩 总体结构梳理

讲义内容可分为以下几大模块：

| 模块 | 对应页码 | 核心库/工具 | 典型任务 |
|------|--------|-----------|--------|
| 1. 科学计算基础 | p.14–p.20 | `NumPy`, `SciPy`, `Matplotlib` | 数值运算、微积分、可视化 |
| 2. 优化问题 | p.20–p.24 | `scipy.optimize` | 最小化目标函数（带约束） |
| 3. 曲线拟合 | p.25–p.34 | `scipy.optimize.curve_fit` | 参数估计（如弹簧振动） |
| 4. 微积分计算 | p.35–p.37 | `sympy`, `scipy.integrate` | 符号/数值微分与积分 |
| 5. 微分方程 | p.38–p.40 | `scipy.integrate.solve_ivp` | 求解 ODE（如带阻力的落体） |
| 6. 时间序列预测 | p.43–p.52 | `statsmodels`, `pandas`, `pytorch` | 负荷预测（ARIMA / LSTM） |

下面逐模块详解。

---

## 🔢 一、科学计算三剑客：NumPy / SciPy / Matplotlib（p.17–p.19）

### ✅ 1. NumPy（Numerical Python）
- **用途**：高效处理同构数组（ndarray）
- **关键对象**：`np.array`
- **常用操作**：
  ```python
  import numpy as np
  x = np.linspace(0, 10, 100)      # 生成 100 个点，从 0 到 10
  y = np.sin(x) * np.exp(-x)       # 向量化运算（无需循环）
  ```

### ✅ 2. SciPy（Scientific Python）
- **子模块**：
  - `scipy.optimize`：优化、拟合
  - `scipy.integrate`：数值积分
  - `scipy.linalg`：线性代数
- **示例**（最小化）：
  ```python
  from scipy.optimize import minimize
  def f(x): return (x[0] - 1)**2 + (x[1] - 2.5)**2
  res = minimize(f, x0=[0, 0])  # x0 是初始猜测
  ```

### ✅ 3. Matplotlib（数据可视化）
- **基本绘图**：
  ```python
  import matplotlib.pyplot as plt
  plt.plot(x, y, label='damped sine')
  plt.xlabel('Time'); plt.ylabel('Amplitude')
  plt.legend(); plt.show()
  ```

---

## ⚙️ 二、优化问题（p.20–p.24）

### 🎯 问题：带约束的最小化
> Minimize \( f(x, y) = (x - 1)^2 + (y - 2.5)^2 \)
> s.t. \( x - 2y + 2 \geq 0 \), \( -x - 2y + 6 \geq 0 \), ...

### ✅ 代码还原（使用 `scipy.optimize.minimize`）

```python
from scipy.optimize import minimize

# 目标函数
def objective(vars):
    x, y = vars
    return (x - 1)**2 + (y - 2.5)**2

# 不等式约束：g(x) >= 0
def constraint1(vars): return vars[0] - 2*vars[1] + 2
def constraint2(vars): return -vars[0] - 2*vars[1] + 6
def constraint3(vars): return -vars[0] + 2*vars[1] + 2

# 定义约束字典（type='ineq' 表示 >=0）
constraints = [
    {'type': 'ineq', 'fun': constraint1},
    {'type': 'ineq', 'fun': constraint2},
    {'type': 'ineq', 'fun': constraint3}
]

# 变量边界（x>=0, y>=0）
bounds = [(0, None), (0, None)]

# 初始猜测
x0 = [0, 0]

# 求解
result = minimize(objective, x0, method='SLSQP', bounds=bounds, constraints=constraints)
print("Optimal x, y:", result.x)
```

### 🔍 关键参数说明：
| 参数 | 说明 |
|------|------|
| `method='SLSQP'` | 支持非线性约束的优化算法 |
| `bounds` | 每个变量的上下界，格式 `[(low1, high1), ...]` |
| `constraints` | 列表，每个元素是 `{'type': 'ineq'/'eq', 'fun': func}` |

---

## 📈 三、曲线拟合（p.25–p.34）

### 🎯 问题 1：拟合 \( y = ax^2 + b \)

```python
from scipy.optimize import curve_fit
import numpy as np

# 假设已有数据 x_data, y_data
def model(x, a, b):
    return a * x**2 + b

popt, pcov = curve_fit(model, x_data, y_data)
a_fit, b_fit = popt
```

### 🎯 问题 2：拟合弹簧振动 \( y(t) = A \cos(\omega t + \phi) \)

```python
def spring_model(t, A, omega, phi):
    return A * np.cos(omega * t + phi)

# 提供初始猜测（讲义 p.32 强调很重要！）
p0 = [4, np.pi, 0]  # A=4, ω=π, φ=0

popt, _ = curve_fit(spring_model, t_data, y_data, p0=p0)
A, omega, phi = popt
natural_freq = omega / (2 * np.pi)  # f = ω / 2π
```

### 🔍 `curve_fit` 关键参数：
| 参数 | 说明 |
|------|------|
| `f` | 模型函数，形式 `f(x, *params)` |
| `xdata, ydata` | 实验数据 |
| `p0` | 初始参数猜测（对非线性拟合至关重要！） |
| 返回 `popt` | 最优参数估计值 |

---

## 🧮 四、微积分计算（p.35–p.37）

### ✅ 1. 符号微分（用 `sympy`）
```python
from sympy import symbols, diff, exp, sin, integrate

x = symbols('x')
f = x**2 * sin(2*x) * exp(-x)
f_prime = diff(f, x)      # 一阶导
f_double = diff(f, x, 2)  # 二阶导
```

### ✅ 2. 数值积分（用 `scipy.integrate.quad`）
```python
from scipy.integrate import quad
import numpy as np

def integrand(x):
    return x**2 * np.sin(2*x) * np.exp(-x)

result, error = quad(integrand, 0, 1)  # ∫₀¹ ...
```

### ✅ 3. 二重积分
```python
from scipy.integrate import dblquad

def integrand(y, x):  # 注意顺序：y 先，x 后
    return np.sin(x + y**2)

# ∫ₓ=0→1 ∫ᵧ=-x→x ...
result, _ = dblquad(integrand, 0, 1, lambda x: -x, lambda x: x)
```

---

## 📉 五、微分方程（p.38–p.40）

### 🎯 问题：带空气阻力的落体
> \( \frac{dv}{dt} = g - k v^2 \), \( v(0) = 0 \)

### ✅ 代码（使用 `solve_ivp`）

```python
from scipy.integrate import solve_ivp
import numpy as np

g = 9.8; k = 0.1

def dvdt(t, v):
    return g - k * v[0]**2  # v 是数组 [v]

t_span = [0, 10]          # 时间区间
t_eval = np.linspace(0, 10, 100)  # 指定输出时间点
v0 = [0]                  # 初始速度

sol = solve_ivp(dvdt, t_span, v0, t_eval=t_eval)

plt.plot(sol.t, sol.y[0])
plt.xlabel('Time (s)'); plt.ylabel('Velocity (m/s)')
```

### 🔍 `solve_ivp` 关键参数：
| 参数 | 说明 |
|------|------|
| `fun` | 导数函数，`fun(t, y)` |
| `t_span` | `[t0, tf]` |
| `y0` | 初始条件（数组） |
| `t_eval` | 指定返回哪些时间点的解 |

---

## 📊 六、时间序列预测 —— ARIMA（p.48–p.50）

### 🎯 任务：电力负荷预测

### ✅ 代码（使用 `statsmodels`）

```python
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

# 假设 data 是 pandas Series，索引为时间
train_size = int(len(data) * 0.8)
train, test = data[:train_size], data[train_size:]

# 拟合 ARIMA(p,d,q) 模型
# 讲义未指定阶数，通常需通过 ACF/PACF 或 auto_arima 确定
model = ARIMA(train, order=(1,1,1))  # 示例阶数
fitted = model.fit()

# 预测未来 len(test) 步
forecast = fitted.forecast(steps=len(test))

# 绘图
plt.plot(train.index, train, label='Train')
plt.plot(test.index, test, label='Actual')
plt.plot(test.index, forecast, label='Forecast')
plt.legend(); plt.show()
```

### 🔍 `ARIMA` 参数说明：
| 参数 | 说明 |
|------|------|
| `order=(p, d, q)` |
| `p` | 自回归项数（AR） |
| `d` | 差分次数（使序列平稳） |
| `q` | 移动平均项数（MA） |

> 💡 实际中常用 `pmdarima.auto_arima()` 自动选择最优 `(p,d,q)`。

---

## 🤖 七、进阶：LSTM（p.51–p.52）

讲义提到：“LSTM is widely used for sequence prediction”。

### ✅ 简化代码框架（PyTorch）

```python
import torch
import torch.nn as nn

class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, output_size=1):
        super().__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # 取最后一个时间步
        return out

# 使用流程：
# 1. 数据预处理（滑动窗口构造 X, y）
# 2. 转为 torch.Tensor
# 3. 训练模型
# 4. 预测
```

> ⚠️ LSTM 需要大量数据预处理（归一化、窗口化）、超参调优，此处仅为示意。

---

## 🧪 八、其他领域简述

| 领域 | 库 | 用途 |
|------|----|------|
| **生物信息学** | `Biopython` | 序列比对（`pairwise2.align.globalxx`） |
| **化学信息学** | `RDKit` | 分子结构可视化、SMILES 解析 |
| **AI for Science** | `AlphaFold` | 蛋白质结构预测（不直接编程，调用 API） |

> 这些在讲义中更多是**应用场景展示**，而非代码教学。

---

## ✅ 总结：核心库与参数速查表

| 任务 | 库 | 关键函数/类 | 重要参数 |
|------|----|------------|--------|
| 数组运算 | `numpy` | `np.array`, `np.linspace` | — |
| 优化 | `scipy.optimize` | `minimize` | `method`, `bounds`, `constraints` |
| 曲线拟合 | `scipy.optimize` | `curve_fit` | `p0`（初始猜测） |
| 积分 | `scipy.integrate` | `quad`, `dblquad` | 积分限、被积函数 |
| ODE 求解 | `scipy.integrate` | `solve_ivp` | `t_span`, `y0`, `t_eval` |
| 时间序列 | `statsmodels` | `ARIMA` | `order=(p,d,q)` |
| 深度学习 | `torch` / `tensorflow` | `LSTM` | `hidden_size`, `num_layers` |
